Multi-resolution registration example

In [ ]:
import os
from itertools import chain

import numpy as np
import SimpleITK as sitk

import itkwidgets
from itkwidgets import view

def itk_image(np_array, resolution):
    img = sitk.GetImageFromArray(np_array.astype(np.float32))
    img.SetOrigin((0, 0, 0))
    img.SetSpacing(resolution)
    return img

def imwrite(image, filename):
    writer = sitk.ImageFileWriter()
    writer.SetFileName(filename)
    writer.Execute(image)
    
def run_elastix(fixed_image, moving_image, params):
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

def print_parameters(elastix_parameters):
    for k,v in elastix_parameters.iteritems():
        print(f'{k} \n\t {v}')


In [ ]:
fixed_image_path='sampleImages/jrc18_down.nrrd'
fixed_image = sitk.ReadImage(fixed_image_path)

In [ ]:
moving_image_path = 'sampleImages/jrc10_down.nrrd'
moving_image = sitk.ReadImage(moving_image_path)

In [ ]:
default_params = sitk.ReadParameterFile('elastixParameters/DefaultBsplineParams.txt')
elastixImageFilter = run_elastix(fixed_image, moving_image, default_params ) 
result_image = elastixImageFilter.GetResultImage()
sitk.WriteImage(result_image, 'defaultbspline_result_img.nrrd')

It takes quite awhile for the above to run, and the result is pretty bad.  It may look okay when zoomed out, but zooming in and scrolling in z will show that there is very high deformation.

In [ ]:
view(result_image)

## Let's make it better



In [ ]:
affine_params = sitk.ReadParameterFile('elastixParameters/DefaultSmoother_Affine.txt')
elastixAffine = run_elastix(fixed_image, moving_image, affine_params ) 
sitk.WriteImage(elastixAffine.GetResultImage(), 'affine_result_img.nrrd')

In [ ]:
# use the affine as the new moving image
affine_result = elastixAffine.GetResultImage()
bspline_params = sitk.ReadParameterFile('elastixParameters/DefaultSmoother_Bspline.txt')
elastixAffineBspline = run_elastix(fixed_image, affine_result, bspline_params ) 
sitk.WriteImage(elastixAffineBspline.GetResultImage(), 'affine_bspline_result_img.nrrd')

A better approach

In [ ]:
parameterMapVector = sitk.VectorOfParameterMap()
parameterMapVector.append(affine_params)
parameterMapVector.append(bspline_params)

elastixParamVector = run_elastix(fixed_image, moving_image, parameterMapVector ) 
sitk.WriteImage(elastixParamVector.GetResultImage(), 'affinebspline_result_img.nrrd')

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(elastixParamVector.GetTransformParameterMap())
transformixImageFilter.SetMovingImage(moving_image)
# transformixImageFilter.SetFixedImage(fixed_image)
transformixImageFilter.Execute()
sitk.WriteImage(transformixImageFilter.GetResultImage(), 'tform_res.nrrd')

In [ ]:
pmap = elastixParamVector.GetTransformParameterMap()
# print_parameters(pmap[1])

pmap[0]['Size'] = ['207', '100', '60']
pmap[0]['Spacing'] = ['3.0', '3.0', '3.0']
pmap[1]['Size'] = ['207', '100', '60']
pmap[1]['Spacing'] = ['3.0', '3.0', '3.0']

In [ ]:

transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(elastixParamVector.GetTransformParameterMap())
transformixImageFilter.SetMovingImage(moving_image)
transformixImageFilter.Execute()

## Bonus - quantifying deformation

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(elastixParamVector.GetTransformParameterMap())
transformixImageFilter.SetMovingImage(moving_image)
transformixImageFilter.SetOutputDirectory('./a')
transformixImageFilter.SetComputeDeterminantOfSpatialJacobian(True)
transformixImageFilter.Execute()

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(elastixImageFilter.GetTransformParameterMap())
transformixImageFilter.SetMovingImage(moving_image)
transformixImageFilter.SetOutputDirectory('./b')
transformixImageFilter.SetComputeDeterminantOfSpatialJacobian(True)
transformixImageFilter.Execute()